# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 29 2022 2:58PM,240777,10,0085813091,ISDIN Corporation,Released
1,Jun 29 2022 2:58PM,240777,10,0085813092,ISDIN Corporation,Released
2,Jun 29 2022 2:58PM,240777,10,0085813093,ISDIN Corporation,Released
3,Jun 29 2022 2:58PM,240777,10,0085813111,ISDIN Corporation,Released
4,Jun 29 2022 2:58PM,240777,10,0085813112,ISDIN Corporation,Released
5,Jun 29 2022 2:58PM,240777,10,0085813113,ISDIN Corporation,Released
6,Jun 29 2022 2:58PM,240777,10,0085814257,ISDIN Corporation,Released
7,Jun 29 2022 2:58PM,240777,10,0085814277,ISDIN Corporation,Released
8,Jun 29 2022 2:58PM,240777,10,0085814275,ISDIN Corporation,Released
9,Jun 29 2022 2:58PM,240777,10,0085814278,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
36,240773,Executing,1
37,240774,Executing,1
38,240776,Released,1
39,240777,Released,28
40,240778,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
240773,NaN,1.0,NaN
240774,NaN,1.0,NaN
240776,NaN,NaN,1.0
240777,NaN,NaN,28.0
240778,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
240665,0.0,0.0,1.0
240671,0.0,0.0,1.0
240672,11.0,16.0,5.0
240679,0.0,0.0,1.0
240680,1.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
240665,0,0,1
240671,0,0,1
240672,11,16,5
240679,0,0,1
240680,1,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,240665,0,0,1
1,240671,0,0,1
2,240672,11,16,5
3,240679,0,0,1
4,240680,1,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,240665,,,1
1,240671,,,1
2,240672,11,16,5
3,240679,,,1
4,240680,1,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 29 2022 2:58PM,240777,10,ISDIN Corporation
28,Jun 29 2022 2:57PM,240778,10,Eywa Pharma Inc.
29,Jun 29 2022 2:56PM,240776,20,Alumier Labs Inc.
30,Jun 29 2022 2:40PM,240774,21,"NBTY Global, Inc."
31,Jun 29 2022 2:37PM,240773,19,"AdvaGen Pharma, Ltd"
32,Jun 29 2022 2:25PM,240772,10,Emerginnova
33,Jun 29 2022 2:25PM,240771,20,Alumier Labs Inc.
39,Jun 29 2022 2:20PM,240770,10,"Nextsource Biotechnology, LLC"
40,Jun 29 2022 2:02PM,240769,19,ACG North America LLC
44,Jun 29 2022 1:53PM,240768,17,"Nextsource Biotechnology, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jun 29 2022 2:58PM,240777,10,ISDIN Corporation,,,28
1,Jun 29 2022 2:57PM,240778,10,Eywa Pharma Inc.,,,1
2,Jun 29 2022 2:56PM,240776,20,Alumier Labs Inc.,,,1
3,Jun 29 2022 2:40PM,240774,21,"NBTY Global, Inc.",,1,
4,Jun 29 2022 2:37PM,240773,19,"AdvaGen Pharma, Ltd",,1,
5,Jun 29 2022 2:25PM,240772,10,Emerginnova,,,1
6,Jun 29 2022 2:25PM,240771,20,Alumier Labs Inc.,,2,4
7,Jun 29 2022 2:20PM,240770,10,"Nextsource Biotechnology, LLC",,,1
8,Jun 29 2022 2:02PM,240769,19,ACG North America LLC,,,4
9,Jun 29 2022 1:53PM,240768,17,"Nextsource Biotechnology, LLC",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 29 2022 2:58PM,240777,10,ISDIN Corporation,28,,
1,Jun 29 2022 2:57PM,240778,10,Eywa Pharma Inc.,1,,
2,Jun 29 2022 2:56PM,240776,20,Alumier Labs Inc.,1,,
3,Jun 29 2022 2:40PM,240774,21,"NBTY Global, Inc.",,1,
4,Jun 29 2022 2:37PM,240773,19,"AdvaGen Pharma, Ltd",,1,
5,Jun 29 2022 2:25PM,240772,10,Emerginnova,1,,
6,Jun 29 2022 2:25PM,240771,20,Alumier Labs Inc.,4,2,
7,Jun 29 2022 2:20PM,240770,10,"Nextsource Biotechnology, LLC",1,,
8,Jun 29 2022 2:02PM,240769,19,ACG North America LLC,4,,
9,Jun 29 2022 1:53PM,240768,17,"Nextsource Biotechnology, LLC",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 29 2022 2:58PM,240777,10,ISDIN Corporation,28,,
1,Jun 29 2022 2:57PM,240778,10,Eywa Pharma Inc.,1,,
2,Jun 29 2022 2:56PM,240776,20,Alumier Labs Inc.,1,,
3,Jun 29 2022 2:40PM,240774,21,"NBTY Global, Inc.",,1,
4,Jun 29 2022 2:37PM,240773,19,"AdvaGen Pharma, Ltd",,1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 29 2022 2:58PM,240777,10,ISDIN Corporation,28.0,NaN,NaN
1,Jun 29 2022 2:57PM,240778,10,Eywa Pharma Inc.,1.0,NaN,NaN
2,Jun 29 2022 2:56PM,240776,20,Alumier Labs Inc.,1.0,NaN,NaN
3,Jun 29 2022 2:40PM,240774,21,"NBTY Global, Inc.",NaN,1.0,NaN
4,Jun 29 2022 2:37PM,240773,19,"AdvaGen Pharma, Ltd",NaN,1.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 29 2022 2:58PM,240777,10,ISDIN Corporation,28.0,0.0,0.0
1,Jun 29 2022 2:57PM,240778,10,Eywa Pharma Inc.,1.0,0.0,0.0
2,Jun 29 2022 2:56PM,240776,20,Alumier Labs Inc.,1.0,0.0,0.0
3,Jun 29 2022 2:40PM,240774,21,"NBTY Global, Inc.",0.0,1.0,0.0
4,Jun 29 2022 2:37PM,240773,19,"AdvaGen Pharma, Ltd",0.0,1.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3129604,59.0,17.0,0.0
12,481389,1.0,1.0,1.0
17,240768,1.0,0.0,0.0
19,1203648,13.0,17.0,11.0
20,1685302,80.0,14.0,0.0
21,481453,1.0,1.0,0.0
22,722150,3.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3129604,59.0,17.0,0.0
1,12,481389,1.0,1.0,1.0
2,17,240768,1.0,0.0,0.0
3,19,1203648,13.0,17.0,11.0
4,20,1685302,80.0,14.0,0.0
5,21,481453,1.0,1.0,0.0
6,22,722150,3.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,59.0,17.0,0.0
1,12,1.0,1.0,1.0
2,17,1.0,0.0,0.0
3,19,13.0,17.0,11.0
4,20,80.0,14.0,0.0
5,21,1.0,1.0,0.0
6,22,3.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,59.0
1,12,Released,1.0
2,17,Released,1.0
3,19,Released,13.0
4,20,Released,80.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,17,19,20,21,22
Status,,,,,,,
Completed,0.0,1.0,0.0,11.0,0.0,0.0,0.0
Executing,17.0,1.0,0.0,17.0,14.0,1.0,0.0
Released,59.0,1.0,1.0,13.0,80.0,1.0,3.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,17,19,20,21,22
0,Completed,0.0,1.0,0.0,11.0,0.0,0.0,0.0
1,Executing,17.0,1.0,0.0,17.0,14.0,1.0,0.0
2,Released,59.0,1.0,1.0,13.0,80.0,1.0,3.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,17,19,20,21,22
0,Completed,0.0,1.0,0.0,11.0,0.0,0.0,0.0
1,Executing,17.0,1.0,0.0,17.0,14.0,1.0,0.0
2,Released,59.0,1.0,1.0,13.0,80.0,1.0,3.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()